# EC Cryptography

![ec](./elliptic-curve-add.png)

Elliptical curve cryptography (ECC) is an area of public key cryptography (not a specific algorithm) based on elliptic curve theory that can be used to create faster, smaller, and more efficient cryptographic keys. ECC generates keys through the properties of the elliptic curve equation instead of the traditional method of generation as the product of very large prime numbers.  Ref: https://uwillnvrknow.github.io/deCryptMe/pages/ellipticCurve.html

The way EC keys are used is similar to RSA with the exception that persistent re-usable keys do not provide for good forward security. So we tend to not encrypt directly with our EC keys. We do use ephemeral EC keys when exchanging a Symmetric key **ECDHE** and when Signing with **ECDSA.**

## ECHDE
EC Diffie Hellman Exhange is a key exchange mechanism that uses ephemeral EC keys to exchange a robust secret for mutual use.

Traditional DH exchange uses the principles used by RSA to share public key components (which are prime exponents) whereas ECDHE exchanges ephemeral (generated just for this purpose) EC public keys (which are just points on an elliptic curve) using the following logic.  These keys are used by each party to create a common key (integer pair) by multiply each others public key with their own private key (the x value of another point on the elliptic curve). EC mathematical magic is such that Alice's (a,b) shares her public key b with Bob (c,d) who in turn creates m=b.c and Bob in turns shares his public key d and Alice creates m=d.a

Now they both have a key not known to anybody else called m, which they may use to exchange further information securely.


> `sudo python3 -m pip install tinyec`

In [15]:
from tinyec import registry
import secrets

# registry.EC_CURVE_REGISTRY



In [17]:
def compress(pubKey):
    return hex(pubKey.x) + hex(pubKey.y % 2)[2:]

curve = registry.get_curve('brainpoolP256r1')

alicePrivKey = secrets.randbelow(curve.field.n)
alicePubKey = alicePrivKey * curve.g
print("Alice public key:", compress(alicePubKey))

bobPrivKey = secrets.randbelow(curve.field.n)
bobPubKey = bobPrivKey * curve.g
print("Bob public key:", compress(bobPubKey))

print("Now exchange the public keys (e.g. through Internet)")

aliceSharedKey = alicePrivKey * bobPubKey
print("Alice shared key:", compress(aliceSharedKey))

bobSharedKey = bobPrivKey * alicePubKey
print("Bob shared key:", compress(bobSharedKey))

print("Equal shared keys:", aliceSharedKey == bobSharedKey)

Alice public key: 0x26a095ef9b588166beb29ae4f27c50f3c6a4a721600e826aa2f49a4f2b76ddf51
Bob public key: 0x9ef72e65d0811174e255ea583e929a047692949c63ca32fdb144dddf38b4cb721
Now exchange the public keys (e.g. through Internet)
Alice shared key: 0x3291272d0850e7a042b9faed711649a9152f24fda48c89f328f2a17a4622ba4b0
Bob shared key: 0x3291272d0850e7a042b9faed711649a9152f24fda48c89f328f2a17a4622ba4b0
Equal shared keys: True


In practice this calculated shared key would then be passed through a key spreading algorithm such as `hkdef` of `pbkdf2` for additional strength and to ensure the key length was always as required.